## Chains


In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import httpx
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0.0,api_key=os.environ['OPENAI_API_KEY'],base_url="https://api-inference.modelscope.cn/v1/", http_client=httpx.Client(verify=False),model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")

In [5]:
import pandas as pd
df = pd.read_csv('Data.csv',encoding='utf-8')

In [6]:
df

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


### 1. LLMChain
LLMChain是一个简单但非常强大的链，也是后面我们将要介绍的许多链的基础。

In [8]:
from langchain.prompts import ChatPromptTemplate   #导入聊天提示模板
from langchain.chains import LLMChain    #导入LLM链。
import warnings
warnings.filterwarnings('ignore')
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(product="Queen Size Sheet Set")

'**QueenCloud**\n\n**Rationale:**\n- **Clarity and Simplicity:** Clearly indicates the product size (Queen) and evokes imagery of comfort and luxury.\n- **Memorable Name:** The name "QueenCloud" is catchy and easy to remember, capturing the essence of sophistication and comfort.\n- **Aesthetic Appeal:** Combines "Queen" with "Cloud," suggesting softness and breathability, which are desirable qualities in bedding.\n- **Potential for Branding:** The name is versatile for branding, with room to incorporate design elements that reflect luxury or style.\n\nThis name effectively communicates the product\'s characteristics while appealing to a premium audience.'

### 2. SimpleSequentialChain
顺序链（Sequential Chains）是按预定义顺序执行其链接的链。具体来说，我们将使用简单顺序链（SimpleSequentialChain），这是顺序链的最简单类型，其中每个步骤都有一个输入/输出，一个步骤的输出是下一个步骤的输入

In [18]:
from langchain.chains import SimpleSequentialChain
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?,only output name"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt)
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{name}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)
simple_chain = SimpleSequentialChain(chains=[chain_one,chain_two],verbose=True)
product = "Queen Size Sheet Set"
simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Luxury Queen Sheets
Experience luxury with our elegant and stylish luxury sheets for your bedroom, crafted with premium materials for unparalleled comfort and sophistication.

> Finished chain.


'Experience luxury with our elegant and stylish luxury sheets for your bedroom, crafted with premium materials for unparalleled comfort and sophistication.'